In [13]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder



In [14]:
# טוען את הדאטה מהתיקייה data/raw
df = pd.read_csv("../data/raw/telco_churn.csv")

# הצגה ראשונית
print(df.shape)
print(df.head())
print(df['Churn'].value_counts())


(7043, 21)
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Co

In [15]:
# --- ניקוי עמודת היעד ---
# לפעמים יש רווחים/אותיות גדולות: ' Yes', 'No '
df["Churn"] = (
    df["Churn"].astype(str).str.strip().str.lower().map({"yes": 1, "no": 0})
)
# אם יש ערכים לא מוכרים, יווצרו NaN; נוודא שאין:
assert df["Churn"].isin([0, 1]).all(), df["Churn"].unique()

# --- תיקוני עמודות ידועות בטלקו (מומלץ) ---
# TotalCharges לפעמים מגיעה כמחרוזת עם רווחים -> המרה למספר
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
    df = df.dropna(subset=["TotalCharges"])

# מזהה לקוח לא שימושי למודל
for col in ["customerID", "customer_id", "CustomerID"]:
    if col in df.columns:
        df = df.drop(columns=[col])

# --- הכנת X,y ---
y = df["Churn"].astype(int)
X = df.drop(columns=["Churn"])

# המרה של קטגוריות לדאמיז לבייסליין מהיר
X = pd.get_dummies(X, drop_first=True)


In [16]:
# מפרידים בין מאפיינים (X) למטרה (y)
X = df.drop("Churn", axis=1)
y = df["Churn"].astype(int)   # המרה ל-0/1 אם צריך

# חלוקה ל-80% אימון ו-20% בדיקה
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [17]:
# זיהוי עמודות נומריות וקטגוריות
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(exclude=["number"]).columns.tolist()


# טרנספורמציות: דאמים לקטגוריות, מעבר שקוף לנומריות
preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    ]
)

# פייפליין: קידוד -> מודל
pipe = Pipeline(
    steps=[
        ("prep", preprocess),
        ("clf", LogisticRegression(max_iter=1000))
    ]
)

# אם עדיין לא עשית split, בטל הערה לשתי השורות הבאות:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# אימון והערכה
pipe.fit(X_train, y_train)
y_pred_proba = pipe.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, y_pred_proba))
print("PR-AUC:", average_precision_score(y_test, y_pred_proba))


ROC-AUC: 0.8363069508363056
PR-AUC: 0.6242470187773215


C:\Users\97254\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
